In [21]:
import matplotlib.pyplot as plt
from matplotlib.patches import RegularPolygon
from matplotlib.lines import Line2D
import pandas as pd
import numpy as np
import re
from collections import defaultdict


print("Complete.")

Complete.


In [32]:
# Configuration
BOARD_DIM = 7  # 7x7 Hex board
HEX_RADIUS = 0.4
HEX_WIDTH = 2 * HEX_RADIUS
HEX_HEIGHT = np.sqrt(3) * HEX_RADIUS


Belonging = "Top 5 Clauses Blue"
BeforeEnd = 0


CLAUSES_TO_VISUALIZE = [
    {
        "Weight": "(Red: -11, Blue: 75)",
        "clause_id": 2991,
        "interpreted": "NOT Node-11_P AND NOT Node-17_C AND NOT Node-30_J AND NOT Node-32_B AND NOT Node-34_O AND NOT Node-38_K AND NOT Node-47_X AND NOT Node-48_W",
    },
    {
        "Weight": "(Red: -18, Blue: 72)",
        "clause_id": 1516,
        "interpreted": "Node-3_A AND NOT Node-0_X AND NOT Node-1_V AND NOT Node-1_Connected AND NOT Node-4_A AND NOT Node-10_R AND NOT Node-11_L AND NOT Node-18_  AND NOT Node-35_B AND NOT Node-35_Y AND NOT Node-37_J AND NOT Node-40_L AND NOT Node-42_E AND NOT Node-47_Z AND NOT Node-48_Not Connected",
    },
    {
        "Weight": "(Red: -48, Blue: 72)",
        "clause_id": 1910,
        "interpreted": "Node-11_K AND NOT Node-1_V AND NOT Node-1_Y AND NOT Node-19_Z AND NOT Node-21_R AND NOT Node-40_A AND NOT Node-42_  AND NOT Node-45_C",
    },
    {
        "Weight": "(Red: -14, Blue: 68)",
        "clause_id": 3327,
        "interpreted": "Node-16_Y AND Node-25_V AND NOT Node-1_Q AND NOT Node-2_  AND NOT Node-4_M AND NOT Node-9_J AND NOT Node-14_K AND NOT Node-18_N AND NOT Node-27_J AND NOT Node-42_A",
    },
    {
        "Weight": "(Red: -55, Blue: 67)",
        "clause_id": 3631,
        "interpreted": "Node-37_G AND NOT Node-1_Y AND NOT Node-4_Connected AND NOT Node-11_B AND NOT Node-14_Y AND NOT Node-16_A AND NOT Node-19_T AND NOT Node-21_  AND NOT Node-21_F AND NOT Node-24_Y AND NOT Node-27_W AND NOT Node-37_C AND NOT Node-44_D AND NOT Node-44_U",
    },
]

print("Complete.")

Complete.


In [33]:
# Colors and styles for occupant features
COLOR_X = "#ff9999"       # Redish for Player 1 (X)
COLOR_O = "#9999ff"       # Bluish for Player 2 (O)
COLOR_EMPTY = "#cccccc"   # Gray for Empty

COLOR_NOT_X = "#d0e0ff"    # Light blue for NOT X
COLOR_NOT_O = "#ffd0d0"    # Light red for NOT O
COLOR_NOT_EMPTY = "#eeeeee" # Light gray for NOT Empty

# Neutral fallback color
NEUTRAL_COLOR = "white"

TEXT_COLOR = "black"

# Colors for connected edges
EDGE_COLOR = {
    "Connected": "red",
    "Not Connected": "blue",
}

FEATURE_MAPPING_CSV = "feature_mapping_7x7_BeforeEnd-0_i-3.csv"


def load_feature_mapping(csv_file):
    df = pd.read_csv(csv_file)
    node_id_to_position = {}
    for _, row in df.iterrows():
        node_id = int(row['Node_ID'])
        position_str = row['Position']
        position = tuple(map(int, re.findall(r'\d+', position_str)))
        node_id_to_position[node_id] = position
    return node_id_to_position

def hex_coords(row, col):
    x = col * (HEX_WIDTH * 0.75)
    y = row * HEX_HEIGHT + (col * (HEX_HEIGHT / 2.0))
    return x, y

def parse_clause(interpreted_expression):
    literal_strs = interpreted_expression.split(" AND ")
    parsed_literals = []
    
    for lit in literal_strs:
        lit = lit.strip()
        neg = False
        if lit.startswith("NOT "):
            neg = True
            lit = lit[4:].strip()
        
        m = re.match(r'Node-(\d+)_(\w*)', lit)
        if m:
            node_id = int(m.group(1))
            feature = m.group(2).strip()
            if not feature:
                feature = "Empty"
            literal_type = "negative" if neg else "positive"
            parsed_literals.append({"node": node_id, "feature": feature, "type": literal_type})
        else:
            print(f"Warning: could not parse literal '{lit}'")
    
    return parsed_literals

def draw_hex_board(ax):
    for i in range(BOARD_DIM):
        for j in range(BOARD_DIM):
            x, y = hex_coords(i, j)
            hex_cell = RegularPolygon(
                (x, y), numVertices=6, radius=HEX_RADIUS, 
                orientation=np.radians(30), edgecolor='gray', 
                facecolor='white', linewidth=1, alpha=0.7
            )
            ax.add_patch(hex_cell)

def draw_black_cross(ax, x, y):
    # Smaller cross
    size = HEX_RADIUS * 0.5
    ax.plot([x - size, x + size], [y - size, y + size], color='black', linewidth=2, zorder=20)
    ax.plot([x - size, x + size], [y + size, y - size], color='black', linewidth=2, zorder=20)

def determine_occupant_representation(literals):
    pos_features = literals["positive"]
    neg_features = [f.replace("NOT ", "") for f in literals["negative"]]

    occupant_pos = None
    for c in ["X", "O", "Empty"]:
        if c in pos_features:
            occupant_pos = c
            break

    occupant_neg = None
    if occupant_pos is None:
        for c in ["X", "O", "Empty"]:
            if c in neg_features:
                occupant_neg = c
                break

    if occupant_pos == "X":
        return (COLOR_X, False, True)
    elif occupant_pos == "O":
        return (COLOR_O, False, True)
    elif occupant_pos == "Empty":
        return (COLOR_EMPTY, False, True)
    
    if occupant_neg == "X":
        return (COLOR_NOT_O, True, True)
    elif occupant_neg == "O":
        return (COLOR_NOT_X, True, True)
    elif occupant_neg == "Empty":
        return (COLOR_NOT_EMPTY, True, True)

    return (None, False, False)

def highlight_literals(ax, parsed_literals, node_id_to_position):
    node_text_data = defaultdict(lambda: {"positive": [], "negative": []})
    for literal in parsed_literals:
        node_id = literal["node"]
        ftype = literal["type"]
        feat = literal["feature"]
        if ftype == "positive":
            node_text_data[node_id]["positive"].append(feat)
        else:
            node_text_data[node_id]["negative"].append("NOT " + feat)

    node_summaries = []

    for node_id, literals in node_text_data.items():
        if node_id not in node_id_to_position:
            continue
        row, col = node_id_to_position[node_id]
        x, y = hex_coords(row, col)

        facecolor, draw_cross, occupant_handled = determine_occupant_representation(literals)
        if not occupant_handled:
            facecolor = NEUTRAL_COLOR

        hex_cell = RegularPolygon(
            (x, y), numVertices=6, radius=HEX_RADIUS,
            orientation=np.radians(30), edgecolor='gray',
            facecolor=facecolor, linewidth=1, zorder=5
        )
        ax.add_patch(hex_cell)

        pos_feats = literals["positive"]
        neg_feats = literals["negative"]
        all_text_lines = pos_feats + neg_feats
        label_str = "\n".join(all_text_lines)

        summary_line = f"Node {node_id}: " + "; ".join(all_text_lines)
        node_summaries.append((node_id, summary_line))

        text_box = dict(boxstyle="round,pad=0.2", facecolor='white', alpha=0.8, edgecolor='none')
        ax.text(
            x, y, label_str,
            fontsize=8, ha='center', va='center', color=TEXT_COLOR,
            zorder=10, bbox=text_box
        )

        if draw_cross:
            draw_black_cross(ax, x, y)

        # Connected/Not Connected edge
        all_features = pos_feats + [f.replace("NOT ", "") for f in neg_feats]
        for feature in all_features:
            if feature in EDGE_COLOR:
                hex_cell.set_edgecolor(EDGE_COLOR[feature])
                hex_cell.set_linewidth(2)
                break

    return node_summaries

def visualize_clause(clause, node_id_to_position):
    parsed_literals = parse_clause(clause["interpreted"])
    fig, ax = plt.subplots(figsize=(12, 10))
    ax.set_title(f"Clause #{clause['clause_id']} \n{Belonging}, BeforeEnd-{BeforeEnd} \n Weight: {clause['Weight']}", fontsize=16, pad=20)
    draw_hex_board(ax)
    node_summaries = highlight_literals(ax, parsed_literals, node_id_to_position)

    max_x, max_y = hex_coords(BOARD_DIM - 1, BOARD_DIM - 1)
    ax.set_xlim(-HEX_WIDTH, max_x + HEX_WIDTH * 1.5)
    ax.set_ylim(-HEX_HEIGHT, max_y + HEX_HEIGHT * 1.5)
    ax.invert_yaxis()
    ax.set_aspect('equal', 'box')
    ax.axis('off')

    plt.subplots_adjust(right=0.8)

    # Legend
    legend_elements = [
        Line2D([0], [0], marker='h', color='w', label='X (Red Player)',
               markerfacecolor=COLOR_X, markersize=15, markeredgecolor='gray'),
        Line2D([0], [0], marker='h', color='w', label='O (Blue Player)',
               markerfacecolor=COLOR_O, markersize=15, markeredgecolor='gray'),
        Line2D([0], [0], marker='h', color='w', label='Empty (Gray)',
               markerfacecolor=COLOR_EMPTY, markersize=15, markeredgecolor='gray'),
        Line2D([0], [0], marker='h', color='w', label='NOT X (Light Blue+Cross)',
               markerfacecolor=COLOR_NOT_X, markersize=15, markeredgecolor='gray'),
        Line2D([0], [0], marker='h', color='w', label='NOT O (Light Red+Cross)',
               markerfacecolor=COLOR_NOT_O, markersize=15, markeredgecolor='gray'),
        Line2D([0], [0], marker='h', color='w', label='NOT Empty (Light Gray+Cross)',
               markerfacecolor=COLOR_NOT_EMPTY, markersize=15, markeredgecolor='gray'),
        Line2D([0], [0], marker='h', color='w', label='Connected',
               markerfacecolor='none', markeredgecolor=EDGE_COLOR['Not Connected'], markersize=15, linewidth=2),
        Line2D([0], [0], marker='h', color='w', label='Not Connected',
               markerfacecolor='none', markeredgecolor=EDGE_COLOR['Connected'], markersize=15, linewidth=2)
    ]

    legend = ax.legend(handles=legend_elements, title="Legend", bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0., frameon=True)
    legend.set_frame_on(True)
    legend.get_frame().set_facecolor('white')
    legend.get_frame().set_edgecolor('gray')

    node_summaries.sort(key=lambda x: x[0])
    sorted_summaries = [line for _, line in node_summaries]
    fig.canvas.draw()
    legend_bb = legend.get_window_extent()
    inv = fig.transFigure.inverted()
    legend_bb_fig = inv.transform(legend_bb)
    gap = 0.02
    x_position = legend_bb_fig[0,0]
    y_position = legend_bb_fig[0,1] - gap

    summary_text = "Node Literals:\n" + "\n".join(sorted_summaries)
    fig.text(x_position, y_position, summary_text, ha='left', va='top', fontsize=10)
    clausess = clause['clause_id']
    weightss = clause['Weight']
    filename = f'Clause_ID-{clausess}_{Belonging}_BeforeEnd-{BeforeEnd}_Weight-{weightss}.png'
    plt.savefig(filename)
    plt.close()

def main():
    node_id_to_position = load_feature_mapping(FEATURE_MAPPING_CSV)
    for clause in CLAUSES_TO_VISUALIZE:
        visualize_clause(clause, node_id_to_position)

if __name__ == "__main__":
    main()
